In [1]:
import os
os.getcwd()
os.chdir("H:\pyspark_advanced-coding_interview")
print(os.getcwd())

from pyspark.sql import SparkSession

# Create a Spark session with optimized settings
spark = (SparkSession.builder .appName("OptimizedLocalSpark") .getOrCreate())
sc = spark.sparkContext

H:\pyspark_advanced-coding_interview


# Reading and processing data from an XML

In [6]:
# Assume you have a DataFrame with an XMLColumn like this:

xml
Copy code


<product>
    <id>101</id>
    <name>Laptop</name>
    <price>1200</price>
    <attributes>
        <color>Silver</color>
        <weight>2kg</weight>
    </attributes>
</product>



SyntaxError: invalid decimal literal (1409042879.py, line 13)

In [10]:
pip install spark-xml

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement spark-xml (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for spark-xml


In [7]:
SparkSession.builder \
    .appName("ReadXMLColumn") \
    .config("spark.jars.packages", "org.apache.spark:spark-xml_2.12:0.15.0") \
    .getOrCreate()


In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType

# Add the spark-xml package
spark = SparkSession.builder \
    .appName("XMLExample") \
    .config("spark.jars.packages", "org.apache.spark:spark-xml_2.12:0.15.0") \
    .getOrCreate()

# Rest of your PySpark code

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_xml, col
from pyspark.sql.types import StructType, StructField, StringType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("ReadXMLColumn") \
    .config("spark.jars.packages", "org.apache.spark:spark-xml_2.12:0.15.0") \
    .getOrCreate()

# Sample DataFrame
data = [
    ("<product><id>101</id><name>Laptop</name><price>1200</price><attributes><color>Silver</color><weight>2kg</weight></attributes></product>",)
]
columns = ["XMLColumn"]
df = spark.createDataFrame(data, columns)

# Define schema for XML
schema = StructType([
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("price", StringType(), True),
    StructField("attributes", StructType([
        StructField("color", StringType(), True),
        StructField("weight", StringType(), True)
    ]))
])

# Parse XML column
df_parsed = df.withColumn("ParsedXML", from_xml(col("XMLColumn"), schema))

# Select specific fields
df_final = df_parsed.select(
    col("ParsedXML.id").alias("ProductID"),
    col("ParsedXML.name").alias("ProductName"),
    col("ParsedXML.price").alias("Price"),
    col("ParsedXML.attributes.color").alias("Color"),
    col("ParsedXML.attributes.weight").alias("Weight")
)

df_final.show(truncate=False)


ImportError: cannot import name 'from_xml' from 'pyspark.sql.functions' (C:\spark\python\pyspark\sql\functions.py)

In [3]:
# Register DataFrame as a SQL table
df.createOrReplaceTempView("XMLTable")

# Query the XML column using Spark SQL
result = spark.sql("""
    SELECT 
        xpath_string(XMLColumn, 'product/id') AS ProductID,
        xpath_string(XMLColumn, 'product/name') AS ProductName,
        xpath_string(XMLColumn, 'product/price') AS Price,
        xpath_string(XMLColumn, 'product/attributes/color') AS Color,
        xpath_string(XMLColumn, 'product/attributes/weight') AS Weight
    FROM XMLTable
""")

result.show(truncate=False)


NameError: name 'df' is not defined

In [4]:
import xml.etree.ElementTree as ET

# Function to parse XML
def parse_xml(xml_string):
    root = ET.fromstring(xml_string)
    return {
        "ProductID": root.find("id").text,
        "ProductName": root.find("name").text,
        "Price": root.find("price").text,
        "Color": root.find("./attributes/color").text,
        "Weight": root.find("./attributes/weight").text
    }

# Example row
xml_data = "<product><id>101</id><name>Laptop</name><price>1200</price><attributes><color>Silver</color><weight>2kg</weight></attributes></product>"

parsed_data = parse_xml(xml_data)
print(parsed_data)


{'ProductID': '101', 'ProductName': 'Laptop', 'Price': '1200', 'Color': 'Silver', 'Weight': '2kg'}


In [5]:
from lxml import etree

# Function to parse XML with XPath
def parse_xml_with_xpath(xml_string):
    root = etree.fromstring(xml_string)
    return {
        "ProductID": root.xpath("id/text()")[0],
        "ProductName": root.xpath("name/text()")[0],
        "Price": root.xpath("price/text()")[0],
        "Color": root.xpath("attributes/color/text()")[0],
        "Weight": root.xpath("attributes/weight/text()")[0]
    }

# Example row
xml_data = "<product><id>101</id><name>Laptop</name><price>1200</price><attributes><color>Silver</color><weight>2kg</weight></attributes></product>"

parsed_data = parse_xml_with_xpath(xml_data)
print(parsed_data)


{'ProductID': '101', 'ProductName': 'Laptop', 'Price': '1200', 'Color': 'Silver', 'Weight': '2kg'}


In [14]:
import xml.etree.ElementTree as ET
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Initialize Spark session
spark = SparkSession.builder.appName("XMLProcessing").getOrCreate()

# Sample DataFrame
data = [
    ("<product><id>101</id><name>Laptop</name><price>1200</price></product>",),
]
columns = ["XMLColumn"]
df = spark.createDataFrame(data, columns)

# Define UDF for parsing XML
def parse_xml(xml_string):
    root = ET.fromstring(xml_string)
    return root.find("id").text

# Register UDF
parse_xml_udf = udf(parse_xml, StringType())

# Apply UDF to extract data from XML
df_parsed = df.withColumn("ProductID", parse_xml_udf(col("XMLColumn")))

df_parsed.show(truncate=False)


+---------------------------------------------------------------------+---------+
|XMLColumn                                                            |ProductID|
+---------------------------------------------------------------------+---------+
|<product><id>101</id><name>Laptop</name><price>1200</price></product>|101      |
+---------------------------------------------------------------------+---------+

